<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/Kondratev_Itog_Analitika_bolshih_dannyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Итоговая работа "Аналитика больших данных"

**Преподаватель:** Алексей Кузьмин

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

**1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы**



Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

**2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок**



Отберем фильмы, которые оценил данный пользователь

**3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь**



Для построения модели нам нужны признаки. В качестве таковых будем использовать:

* Год выхода

* Жанры
* Общее количество оценок

* Суммарную оценку




**4. Добавьте к датафрейму из задания 3 столбцы:**

  * По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

  * столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей



Теперь все готово и можно строить модель


**5. Сформируйте X_train, X_test, y_train, y_test**

**6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах**

**7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии**


Вторая часть практики Python связана со Spark’ом

**8. Загрузить данные в spark**

**9. Средствами спарка вывести среднюю оценку для каждого фильма**

**10. Посчитайте средствами спарка среднюю оценку для каждого жанра**

**11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)**


## Решение

### 1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [36]:
import pandas as pd

# Ссылка на файл с оценками
ratings_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.data.csv"

# Загружаем файл с оценками в датафрейм
ratings_df = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# Ссылка на файл с фильмами
movies_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.item.csv"

# Загружаем файл с фильмами в датафрейм
movies_df = pd.read_csv(movies_url, sep='|', encoding='latin-1', header=None,
                       names=['movie_id', 'movie_title', 'release_date', 'video_release_date','IMDb_URL',
                              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                              'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                              'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [37]:
# Посмотрим на первые строки каждого датафрейма
ratings_df.head(3)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


```
u.data — полный набор данных u, 100 000 оценок 943 пользователей по 1682 элементам.
```
```
Каждый пользователь оценил не менее 20 фильмов. Пользователи и элементы
пронумерованы последовательно с 1. Данные случайным образом
заказал.
```

```
Столбцы:

user_id	                          item_id	      rating	      timestamp
идентификатор пользователя | идентификатор элемента | рейтинг | временная метка
```
```
Это список с разделителем табуляция sep='\t'
```
```
Временные метки представляют собой unix-секунды с 01.01.1970 UTC.
```

In [39]:
movies_df.head(3)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


```
u.item - Информация фильмах:
```

```
Столбцы:

идентификатор фильма | название фильма | дата выпуска | дата выхода видео |
URL-адрес IMDb | неизвестный | Действие | Приключения | Анимация |
Детская | комедия | Преступность | Документальный | Драма | Фэнтези |
Фильм-Нуар | Ужасы | Музыкальный | Тайна | Романтика | Научная фантастика |
Триллер | Война | Западная |
```           
```
Это список с разделителем sep='|'
```
```
Последние 19 полей — жанры,

1 — фильм. относится к этому жанру

0 - означает, что это не так

фильмы могут быть внесколько жанров одновременно
```
```
Идентификаторы фильмов — это те, которые используются в наборе данных u.data.
```

### 2. Средствами Pandas, используя dataframe `ratings`, найдите id пользователя, поставившего больше всего оценок

In [57]:
# Группируем данные по 'user_id' и считаем количество оценок каждого пользователя
# По умолчанию метод `value_counts()` в Pandas сортирует уникальные значения в порядке убывания количества их появлений.
user_ratings_count = ratings_df['user_id'].value_counts()

print(f"ID пользователя с наибольшим количеством оценок: {user_ratings_count.index[0]}")
print(f"Количество оценок пользователя: {user_ratings_count.values[0]}")


ID пользователя с наибольшим количеством оценок: 405
Количество оценок пользователя: 737
